In [1]:
dataset = open('dataset.csv', 'r', encoding="utf8").read().split('\n')

for i in range(1, len(dataset) - 1):
    textV = list(dataset[i])
    textV[-2] = '§'
    dataset[i] = ''.join(textV)

for i in range(len(dataset)):
    dataset[i] = dataset[i].split('§')

dataset.pop(0)
dataset[0][1]

'0'

In [2]:
data = [i[0] for i in dataset]
labels = [i[1] for i in dataset[:-1]]
print(data[0])
labels[0]

"Former Florida Gators quarterback Tim Tebow waves as he stands on the sidelines before the Gators play against the Louisville Cardinals in the 2013 Allstate Sugar Bowl NCAA football game in New Orleans, Louisiana January 2, 2013. Former NFL quarterback and Heisman Trophy winner Tim Tebow has announced the opening of the Tebow CURE Hospital in Davao City, Philippines, which will serve the critical orthopedic needs of children who are not fortunate enough to afford treatments and simple procedures. The Tim Tebow Foundation announced in a press release that the hospital, which has been in the works since 2011, officially received its operating license from the health department last week and is already caring for impoverished children. However, the hospital's grand opening will not happen until spring 2015. The hospital was built in partnership with the Christian organization CURE International, which is a non-profit devoted to the medical care of suffering children. The five-story facil

'0'

In [3]:
import torch
torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'

D:\PythonEnviroments\Torchenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'NVIDIA GeForce GTX 1050 Ti'

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('Algorithms\\fake-news-heatmap-creator')
model = AutoModelForSequenceClassification.from_pretrained('Algorithms\\fake-news-heatmap-creator')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
device

device(type='cuda')

In [5]:
import torch.nn.functional as F
import pandas as pd

SIZE = 300
MAX_TRAIN_LENTH = 400

ds = pd.DataFrame({'text':[], 'prediction':[], 'label':[]})

for i in range(len(labels)):
    words = data[i].split(' ')
    words = [*words, *words[:SIZE]]
    windows = []
    for j in range(len(words) - SIZE):
        windows.append(' '.join(words[j:j + SIZE]))
    batch = tokenizer(windows, padding='max_length', truncation=True, return_tensors='pt', max_length=MAX_TRAIN_LENTH)
    batch.to(device)
    with torch.no_grad():
        outputs = model(**batch)
        predictions = F.softmax(outputs.logits, dim=1)
        predictions = torch.argmax(predictions, dim=1)
        ds = pd.concat([ds,pd.DataFrame({
            'text':windows, 
            'prediction':predictions, 
            'label':[int(labels[i])]*len(windows)
            })
        ])
    break

ds

RuntimeError: CUDA out of memory. Tried to allocate 746.00 MiB (GPU 0; 4.00 GiB total capacity; 3.33 GiB already allocated; 0 bytes free; 3.38 GiB reserved in total by PyTorch)